In [21]:
import numpy as np 
import pandas as pd
import sys
from astropy.io import fits
import plotly.graph_objects as go
sys.path.append('../../dash_gaia_orbits/')

# Package imports
from dash_gaia_orbits.dash_test import create_dash_app
from dash_gaia_orbits import StarClusterData, StarClusterCollection, StarClusters3DPlotter

column_renaming_dict = {'n_stars_hunt' : 'n_stars', 'U_new' : 'U', 'V_new' : 'V', 'W_new' : 'W', 'x_hunt_50' : 'x', 'y_hunt_50' : 'y', 'z_hunt_50' : 'z', 'U_err_new' : 'U_err', 'V_err_new' : 'V_err', 'W_err_new' : 'W_err'}

In [22]:
static_traces = []
static_traces_times = []


# read in Gordian scatter map
dfv = pd.read_csv('/Users/cam/Downloads/edenhofer_dust_scatter_new.csv')
#dfv = dfv.loc[dfv['extinction'].between(0.0003, 1)]
dfv = dfv.loc[dfv['extinction'].between(0.0007, 1)]
ds_index = 2
scatter_edenhofer = go.Scatter3d(
    x=dfv['X'].values[::ds_index],
    y=dfv['Y'].values[::ds_index],
    z=dfv['Z'].values[::ds_index],
    mode='markers',
    marker=dict(size=5.,
                color='gray',
                # color=dfv['extinction'].values[::ds_index],
                # colorscale='Gray_r',
                cmin = 0.0007,
                cmax = 0.005,
                symbol='circle',
                opacity=.01),
    line = dict(color = 'gray', width = 0.),
    name='Edenhofer Dust',
    visible = True,
    hovertext='Edenhofer Dust',
    hoverinfo='skip'
    )
static_traces.append(scatter_edenhofer)
static_traces_times.append([0.]) # only show at t=0


# read in leike map






hdu = fits.open('/Users/cam/Downloads/Local_Bubble_Pelgrims2020.fits')
x = hdu[1].data['x'][::4]
y = hdu[1].data['y'][::4]
z = hdu[1].data['z'][::4]

scatter_lb = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(size=1,
                color='#6495ED',
                symbol='circle',
                opacity=.05),
    name='Local Bubble',
    visible = 'legendonly',
    hovertext='Local Bubble')
static_traces.append(scatter_lb)
static_traces_times.append([0.]) # only show at t=0

In [23]:
df = pd.read_csv('/Users/cam/Downloads/lacerta_members_3dvelocities_new.csv')
df_med = df.groupby('label').median().reset_index()
#df_med['age_myr'] = 10**df_med['age_myr']/1e6
df_med = df_med.rename(columns={'x_helioc' : 'x', 'y_helioc' : 'y', 'z_helioc' : 'z', 'u_hel' : 'U', 'v_hel' : 'V', 'w_hel' : 'W', 'label' : 'name'})
df_med = df_med.drop(columns=['source_id'])
#df_med = df_med.loc[df_med['name'] != 4]


In [24]:
df_hunt_gagne = pd.read_csv('/Users/cam/Downloads/cluster_sample_data.csv')
df_hunt_gagne = df_hunt_gagne[['name', 'family', 'age_myr', 'U', 'V', 'W', 'x_helio', 'y_helio', 'z_helio', 'n_stars']]
df_hunt_gagne = df_hunt_gagne.rename(columns={'x_helio' : 'x', 'y_helio' : 'y', 'z_helio' : 'z'})

sun = pd.DataFrame({'name' : ['Sun'], 'x' : [0], 'y' : [0], 'z' : [0], 'U' : [0], 'V' : [0], 'W' : [0], 'age_myr' : [4600], 'n_stars' : [1]})

In [25]:
lacerta = StarClusterData(df_med, data_name = 'Lacerta clusters', min_size=0, max_size = 10, color = 'violet')
clusters = StarClusterData(df_hunt_gagne, data_name = 'Hunt & Gagne clusters', min_size=0, max_size = 5, color = 'gray', opacity = 1.)
sun_data = StarClusterData(sun, data_name = 'Sun', min_size = 0, max_size = 7, color = 'yellow', opacity = 1., marker_style = 'circle')
sc = StarClusterCollection([lacerta, clusters, sun_data])

In [26]:
xyz_widths = (400, 400, 300)
plot_3d = StarClusters3DPlotter(
    data_collection = sc, 
    xyz_widths = xyz_widths, 
    figure_theme = 'dark', 
    trace_grouping_dict=None)


save_name = '/Users/cam/Downloads/lacerta_mate_zoom.html'
#save_name = '/Users/cam/Downloads/gamma_vel_movie.html'
time_int = np.round(np.arange(0., -40.5, -0.5), 1)
#time_int = np.round(np.arange(0., -20.1, -0.1), 1)
scatter_edenhofer['marker']['color'] = 'gray'
scatter_edenhofer['marker']['size'] = 15
scatter_edenhofer['marker']['opacity'] = 0.02

fig3d_m6 = plot_3d.generate_3d_plot(
    time = time_int,
    show = False, 
    save_name =save_name, 
    static_traces = static_traces, 
    static_traces_times = static_traces_times, 
    static_traces_legendonly=False,
    focus_group = 'Lacerta clusters',
    fade_in_time = 10, # Myr,
    fade_in_and_out = False
)

In [27]:
# xyz_widths = (1300, 1300, 300)
# plot_3d = StarClusters3DPlotter(
#     data_collection = sc, 
#     xyz_widths = xyz_widths, 
#     figure_theme = 'dark', 
#     trace_grouping_dict=None)


# save_name = '/Users/cam/Downloads/lacerta_mate.html'
# #save_name = '/Users/cam/Downloads/gamma_vel_movie.html'
# time_int = np.round(np.arange(0., -40.5, -0.5), 1)
# #time_int = np.round(np.arange(0., -20.1, -0.1), 1)
# scatter_edenhofer['marker']['color'] = 'white'
# scatter_edenhofer['marker']['opacity'] = 0.004

# fig3d_m6 = plot_3d.generate_3d_plot(
#     time = time_int,
#     show = False, 
#     save_name =save_name, 
#     static_traces = static_traces, 
#     static_traces_times = static_traces_times, 
#     static_traces_legendonly=False,
#     focus_group = None,
#     fade_in_time = 10, # Myr,
#     fade_in_and_out = False
# )